### How we can handle missing Values: 

In [2]:
import numpy as np
import pandas as pd
from __future__ import division
pd.set_option('display.width',5000)

In [28]:
import requests
url = 'http://www.boxofficemojo.com/movies/?id=monstersinc2.htm'
response = requests.get(url)
page = response.text

from bs4 import BeautifulSoup
soup=BeautifulSoup(page)

import re

def get_movie_value(soup, field_name):
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return None
    
budget=get_movie_value(soup,'Production Budget')
print budget

N/A


In [3]:
data=pd.read_csv('/Users/username/nyc16_ds8/challenges/challenges_data/2013_movies.csv')

In [4]:
data[5:10]

,Title,Budget,DomesticTotalGross,Director,Rating,Runtime,ReleaseDate
5,Gravity,100000000,274092705,Alfonso Cuaron,PG-13,91,10/4/13 0:00
6,Monsters University,"""N/A""",268492764,Dan Scanlon,G,107,6/21/13 0:00
7,The Hobbit: The Desolation of Smaug,"""N/A""",258366855,Peter Jackson,PG-13,161,12/13/13 0:00
8,Fast & Furious 6,160000000,238679850,Justin Lin,PG-13,130,5/24/13 0:00
9,Oz The Great and Powerful,215000000,234911825,Sam Raimi,PG,127,3/8/13 0:00


In [5]:
data.Budget=data.Budget.replace('"N/A"',np.nan)

# because there were strings. . entire column was read as string
data.Budget=data.Budget.astype(float)
data[5:10]

,Title,Budget,DomesticTotalGross,Director,Rating,Runtime,ReleaseDate
5,Gravity,100000000,274092705,Alfonso Cuaron,PG-13,91,10/4/13 0:00
6,Monsters University,NaN,268492764,Dan Scanlon,G,107,6/21/13 0:00
7,The Hobbit: The Desolation of Smaug,NaN,258366855,Peter Jackson,PG-13,161,12/13/13 0:00
8,Fast & Furious 6,160000000,238679850,Justin Lin,PG-13,130,5/24/13 0:00
9,Oz The Great and Powerful,215000000,234911825,Sam Raimi,PG,127,3/8/13 0:00


In [6]:
data.describe()

,Budget,DomesticTotalGross,Runtime
count,8.900000e+01,1.000000e+02,100.000000
mean,7.474719e+07,1.005969e+08,112.260000
std,5.941692e+07,8.739641e+07,18.190696
min,2.500000e+06,2.556825e+07,75.000000
25%,2.800000e+07,4.270413e+07,98.000000
50%,5.500000e+07,6.954237e+07,112.000000
75%,1.100000e+08,1.204759e+08,123.000000
max,2.250000e+08,4.246680e+08,180.000000


In [7]:
# Which columns have missing values ? 
# what percentage is missing ? 
for i in data.columns:
    print i, (data[i].isnull().sum())/len(data)

Title 0.0
Budget 0.11
DomesticTotalGross 0.0
Director 0.04
Rating 0.0
Runtime 0.0
ReleaseDate 0.0


In [8]:
# Fillna ??  we could but,

# data.Budget=data.Budget.fillna(data.Budget.mean())


In [9]:
# Explore patterns before filling data ! 

data_gb=data.groupby(['Rating']).sum()
data_gb.head()

,Budget,DomesticTotalGross,Runtime
Rating,,,
G,NaN,268492764,107
PG,1409000000,1967035741,1499
PG-13,3961000000,5238139144,5523
R,1282500000,2586019873,4097


In [10]:
# Some algorithms require that we infer missing values ..
# We can use Regression for this

data['Intercept']=np.ones(len(data))

y_train=data.Budget[data.Budget.isnull()==False]
# what we're trying to predict
y_test=data.Budget[data.Budget.isnull()==True]

x_train=data[['Intercept','DomesticTotalGross','Runtime']][data.Budget.isnull()==False]
x_test=data[['Intercept','DomesticTotalGross','Runtime']][data.Budget.isnull()==True]



In [12]:
data2=data
import statsmodels.formula.api as sm
model = sm.OLS(y_train,x_train )
results = model.fit()

for i in y_test.index:
    data2.Budget.loc[i]=results.predict(x_test.loc[i])  
    


In Summary : 

1)  Avoid deleting your entire row of data.  
2)  We can impute the missing value using different algorithms:  
(Regression, KNN).  If we choose to impute:  
    - it's important to be sure that your data is 'truly missing at random'.   
    - The value we are inferring is not really adding anything to the model  
3)  If your algorithm permits missing data, then go with it!  

In [ ]:
# Other packages: 

# https://pypi.python.org/pypi/fancyimpute/0.0.4

